In [1]:
from docplex.mp.model import Model
import numpy as np
from source_torch.mlca.mlca_nn import MLCA_NN
from source_torch.mlca.mlca_nn_mip import MLCA_NNMIP
import pandas as pd
import torch
import docplex.mp.conflict_refiner as cr

MLCA NN Class imported
MLCA NN_MIP Class imported


In [2]:
def _get_model_weights(model): # torch
    nnmodel = model.model
    weights = []
    for params in nnmodel.parameters():
        weights.append(params.detach().cpu().numpy().T)        
    return weights

def _get_model_layer_shapes(model, layer_type=None):
    ''' return layer output shapes instead, 
        if 'input' is given as desired layer type, insert input dim at the beginning.
        assumes torch model '''
    # nnmodel = self.Models[key]
    nnmodel = model.model
    Layer_shapes = []
    for i, (name, param) in enumerate(nnmodel.named_parameters()):
        if (i==0) and ('input' in layer_type): 
            Layer_shapes.append(param.shape[1])
        if any([x in name for x in layer_type]) and ('bias' in name):
            Layer_shapes.append(param.shape[0])
    return Layer_shapes

def _clean_weights(Wb):
    for v in range(0, len(Wb)-2, 2):
        Wb[v][abs(Wb[v]) <= 1e-8] = 0
        Wb[v+1][abs(Wb[v+1]) <= 1e-8] = 0
        zero_rows = np.where(np.logical_and((Wb[v] == 0).all(axis=0), Wb[v+1] == 0))[0]
        if len(zero_rows) > 0:
            #logging.debug('Clean Weights (rows) %s', zero_rows)
            Wb[v] = np.delete(Wb[v], zero_rows, axis=1)
            Wb[v+1] = np.delete(Wb[v+1], zero_rows)
            Wb[v+2] = np.delete(Wb[v+2], zero_rows, axis=0)
    return(Wb)

In [3]:
device = torch.device("cpu")
np.random.seed(0)
torch.manual_seed(0)

bundles = np.random.randint(2, size=(5, 10)).tolist()
values = np.random.randint(100, size=(5, 1)).flatten().tolist()

model = MLCA_NN(X_train = np.array(bundles), Y_train=np.array(values))
model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'regularization_type': 'l2', 'regularization': 0.01}, device= device)

#big integer
C = 6000
L = 6000

count = 1
while count > 0:
    count -= 1

    model.X_train = np.array(bundles)
    model.Y_train = np.array(values)
    model.fit(epochs=100, batch_size=32)

    weights = _get_model_weights(model)
    layers = _get_model_layer_shapes(model, layer_type=['dense', 'input'])
    Wb = _clean_weights(weights)
    W = Wb[0]
    bias = Wb[1]

    n = len(bundles)
    num_items = len(bundles[0])

    vectors = []
    distances = []

    for i in range(n):
        m  = Model(name='galo_nn')
        z = {}
        y = {}
        q = {}
        r = m.continuous_var(name='r')
        x = m.binary_var_list(range(num_items), name='x')
        b = m.binary_var_list(range(n), name='b')
        l_layer = len(layers)-1
        for layer, neurons in enumerate(layers):
            for neuron in range(neurons):
                z[layer, neuron] = m.continuous_var(name='z_{}_{}'.format(str(layer), str(neuron)))
                y[layer, neuron] = m.binary_var(name='y_{}_{}'.format(str(layer), str(neuron)))
                q[layer, neuron] = m.continuous_var(name='q_{}_{}'.format(str(layer), str(neuron)))
        
        for neuron in range(num_items): 
            m.add_constraint(z[0, neuron] == x[neuron])

        m.add_constraint(values[i] - z[l_layer, 0] <= r)
        m.add_constraint(z[l_layer, 0] - values[i] <= r)
        
        for j in range(n):
            m.add_constraint(x[j] <= 1)
            m.add_constraint(b[j] <= 1)
            m.add_constraint(values[j] - z[l_layer, 0] + C * b[j] >= r)
            m.add_constraint(z[l_layer, 0] - values[j] + C * (1- b[j]) >= r)

        for layer, neurons in enumerate(layers):
            if layer == 0: continue
            for neuron in range(neurons):
                if z[layer, neuron] is z[l_layer, 0]:
                    print('it stops here')
                    break
                m.add_constraint(z[layer, neuron] - q[layer, neuron] == W[layer-1][neuron] * z[layer-1, neuron] + bias[layer-1])
                m.add_constraint(0 <= z[layer, neuron])
                m.add_constraint(z[layer, neuron] <= y[layer, neuron] * L)                
                m.add_constraint(0 <= q[layer, neuron])
                m.add_constraint(q[layer, neuron] <=(1 - y[layer, neuron]) * L)

        m.maximize(r)
        m.solve()
        res = m.get_solve_status()
        print(res)
        # if res.name == 'INFEASIBLE_SOLUTION':  # or also 'INFEASIBLE_OR_UNBOUNDED_SOLUTION'
        #     cref = cr.ConflictRefiner()
        #     print('show some of the constraints that can be removed to arrive at a minimal conflict')
        #     confl = cref.refine_conflict(m, display=True)  # display flag is to show the conflicts
        #     #print('Initial y')
        #     #print(values[bundle])
        #     #print('Intercept')
        #     #print(intercept)
        #     cref.display_conflicts(confl)
        c_bundle = [x[k].solution_value for k in range(num_items)]
        vectors.append(c_bundle)
        distances.append(r.solution_value)
        print(c_bundle)
        print(r.solution_value)
    # print('Here is all r values and bundles')
    # print(distances)
    # print(vectors)
    # print('Here is the biggest distance')
    # print(np.argmax(distances))
    # print('Here is the bundle to add')
    #print(vectors[np.argmax(distances)])
    bundles.append(vectors[np.argmax(distances)])
    values.append(np.random.randint(1,100))
# print(bundles)
# print(values)
#free cuda cache


it stops here
JobSolveStatus.OPTIMAL_SOLUTION
[0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
12.5
it stops here
JobSolveStatus.OPTIMAL_SOLUTION
[0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
2972.0
it stops here
JobSolveStatus.OPTIMAL_SOLUTION
[0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
11.5
it stops here
JobSolveStatus.OPTIMAL_SOLUTION
[0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
12.5
it stops here
JobSolveStatus.OPTIMAL_SOLUTION
[0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
12.5
